In [1]:
import copy, os
import math
import numpy as np
import pandas as pd
import scipy.optimize as opt
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

In [2]:
drug_block_df = pd.read_csv( "./Hill_fitting/data/drug_block.csv")
drug_block_df.head()

,drug,conc,units,channel,block,pacing
0,amiodarone,0.8,nM,ICaL,0.0,0.1
1,amiodarone,0.8,nM,ICaL,1.9,0.1
2,amiodarone,0.8,nM,ICaL,0.3,0.1
3,amiodarone,80.0,nM,ICaL,17.4,0.1
4,amiodarone,80.0,nM,ICaL,12.9,0.1


In [3]:
hill_result_dir = "./Hill_fitting/results"
drug_li = os.listdir(hill_result_dir)
drug_li               

['bepridil',
 'chlorpromazine',
 'cisapride',
 'diltiazem',
 'dofetilide',
 'mexiletine',
 'ondansetron',
 'quinidine',
 'ranolazine',
 'sotalol',
 'terfenadine',
 'verapamil']

In [4]:
def get_Gfc(conc, IC50, h):
    return 1.0+(conc/10**np.log10(IC50))**h

In [5]:
ions = ["INa", "INaL", "Ito", "ICaL", "IKs", "IK1", 'hERG',]
optimized = True

for ion in ions :
    
    gfc_li = []    
    
    for drug in drug_li : 
        
        drug_ion_df = drug_block_df[(drug_block_df['drug']==drug) & (drug_block_df['channel']==ion)]        
                 
        IC50_df = pd.read_csv(f"{hill_result_dir}/{drug}/IC50_optimal.csv")    
        IC50 = IC50_df[f"{ion}_IC50"].item()
        h =  IC50_df[f"{ion}_h"].item()    
        if math.isnan(IC50) or math.isnan(h):        
            continue        
        gfcs = get_Gfc(drug_ion_df['conc'], IC50, h)                                        
                
        if optimized :
            if ion=="INaL":            
                gfcs = gfcs/2.661
            elif ion=="ICaL":
                gfcs = gfcs/1.007
            elif ion=="hERG":
                gfcs = np.full(gfcs.shape, 1.1)                
                gfcs = gfcs/1.114
            elif ion=="IKs":
                gfcs = gfcs/1.870
            elif ion=="Ik1":
                gfcs = gfcs/1.698
                       
        
        gfc_li.append(gfcs.min())
        gfc_li.append(gfcs.max())
        
    gfc_li = np.array(gfc_li)    
    print(f"Gfc of {ion} - min: {gfc_li.min()}  |  max: {gfc_li.max()}")

Gfc of INa - min: 1.0000610032568693  |  max: 6.335927561138344
Gfc of INaL - min: 0.37665083057147214  |  max: 1.4094220816010146
Gfc of Ito - min: 1.0000000019929693  |  max: 2.7518842312791723
Gfc of ICaL - min: 0.9945895580245948  |  max: 29.777326113370542
Gfc of IKs - min: 0.5351896228942729  |  max: 1.1454125735490408
Gfc of IK1 - min: 1.0015305275392006  |  max: 2.0894732182411744
Gfc of hERG - min: 0.9874326750448833  |  max: 0.9874326750448833
